In [10]:
import us
import pathlib
import pandas as pd
import geopandas as gpd

In [2]:
csv_list = [f"/media/palewire/Passport/htc/processed/acs5_2017_internet_tracts_{s.abbr.lower()}.csv" for s in us.STATES]

In [3]:
df_list = []

In [5]:
for csv in csv_list:
    df = pd.read_csv(csv, dtype={"geoid": str})
    df_list.append(df)

In [6]:
df = pd.concat(df_list)

In [7]:
df.head()

,geoid,name,total_households,total_with_computer,dialup_internet,broadband_internet,computer_no_internet,total_no_computer,county,state,tract,total_with_internet,total_no_internet
0,01007010001,"Census Tract 100.01, Bibb County, Alabama",915.0,619.0,17.0,565.0,37.0,296.0,7,1,10001,582.0,333.0
1,01007010002,"Census Tract 100.02, Bibb County, Alabama",1981.0,1536.0,4.0,1230.0,302.0,445.0,7,1,10002,1234.0,747.0
2,01007010003,"Census Tract 100.03, Bibb County, Alabama",1729.0,1419.0,0.0,1212.0,207.0,310.0,7,1,10003,1212.0,517.0
3,01007010004,"Census Tract 100.04, Bibb County, Alabama",2291.0,1602.0,0.0,1263.0,339.0,689.0,7,1,10004,1263.0,1028.0
4,01087231900,"Census Tract 2319, Macon County, Alabama",634.0,464.0,15.0,387.0,62.0,170.0,87,1,231900,402.0,232.0


In [8]:
df.to_csv("./data/analysis/internet-by-tract.csv", index=False)

In [11]:
tracts = gpd.read_file("/media/palewire/Passport/gentrified-la/us.shp")

In [12]:
merged = tracts.merge(
    df,
    left_on="GEOID10",
    right_on="geoid",
    how="inner"
)

In [13]:
len(merged)

73031

In [14]:
merged.head()

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,...,total_with_computer,dialup_internet,broadband_internet,computer_no_internet,total_no_computer,county,state,tract,total_with_internet,total_no_internet
0,01,005,950300,01005950300,9503,Census Tract 9503,G5020,S,352811329,499970,...,368.0,14.0,260.0,94.0,323.0,5,1,950300,274.0,417.0
1,01,005,950900,01005950900,9509,Census Tract 9509,G5020,S,16201446,8339342,...,1190.0,6.0,1026.0,158.0,362.0,5,1,950900,1032.0,520.0
2,01,005,950800,01005950800,9508,Census Tract 9508,G5020,S,14630162,380622,...,821.0,9.0,651.0,161.0,117.0,5,1,950800,660.0,278.0
3,01,005,950700,01005950700,9507,Census Tract 9507,G5020,S,127200997,112809,...,451.0,12.0,360.0,79.0,179.0,5,1,950700,372.0,258.0
4,01,005,950600,01005950600,9506,Census Tract 9506,G5020,S,101697268,1101261,...,622.0,27.0,479.0,116.0,134.0,5,1,950600,506.0,250.0


In [18]:
trimmed = merged[[
    'geoid',
    'total_households',
    'total_with_internet',
    'geometry'
]]

In [19]:
trimmed['percent_with_internet'] = (
    trimmed.total_with_internet / trimmed.total_households
)

/home/palewire/.virtualenvs/native-american-demographics-pO-dAzed/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
trimmed.to_file("/media/palewire/Passport/htc/tracts-with-internets.shp")